In [55]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random
from scipy.optimize import curve_fit

import h5py
from IPython.utils import io

from joblib import Parallel, delayed
from tqdm import tqdm
import pandas as pd

In [2]:
%matplotlib widget

In [3]:
%reload_ext autoreload
%autoreload 2

In [8]:
import sys
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
import BeadDataFile
from discharge_tools import *

from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

sys.path.append('/home/analysis_user/New_trap_code/SensitivityFramework/')

### scale

In [11]:
# 5um
bdf_discharge = load_dir(dirname='/data/new_trap/20201116/Bead2/Discharge/', 
                         start_file=0, max_file=175)

2674  files in folder
175  files loaded


In [198]:
bandwidth = 2
m2 = gfw.build_z_response(bdf_discharge, drive_freq=71, charges=1, bandwidth=bandwidth, bimodal=False)

***************************************************
Z2-amplitude:  1.00e+04
reduced chi2:  7206169967.157797
***************************************************
Z2-amplitude:  1.00e+04
reduced chi2:  8534858087.099429
***************************************************
Z2-amplitude:  1.00e+04
reduced chi2:  8666497718.870903
***************************************************
Z2-amplitude:  1.00e+04
reduced chi2:  8877686841.0249
***************************************************
Z2-amplitude:  1.00e+04
reduced chi2:  5866032127.755403
***************************************************
Z2-amplitude:  1.00e+04
reduced chi2:  435926595.10218817
***************************************************
Z2-amplitude:  3.94e+02
reduced chi2:  7667.699630606357
***************************************************
Z2-amplitude:  4.28e+02
reduced chi2:  7761.9207376647355
***************************************************
Z2-amplitude:  9.79e+03
reduced chi2:  66901.9834298937
********************

***************************************************
Z2-amplitude:  1.54e+03
reduced chi2:  5888.995202619574
***************************************************
Z2-amplitude:  1.53e+03
reduced chi2:  6771.667471997413
***************************************************
Z2-amplitude:  1.52e+03
reduced chi2:  7957.565796601961
***************************************************
Z2-amplitude:  1.58e+03
reduced chi2:  3999.069001165491
***************************************************
Z2-amplitude:  1.56e+03
reduced chi2:  7749.5383518738045
***************************************************
Z2-amplitude:  1.52e+03
reduced chi2:  6815.433269858348
***************************************************
Z2-amplitude:  1.53e+03
reduced chi2:  5223.081869842338
***************************************************
Z2-amplitude:  1.53e+03
reduced chi2:  4394.563952584297
***************************************************
Z2-amplitude:  1.53e+03
reduced chi2:  4917.521102582212
******************

***************************************************
Z2-amplitude:  3.13e+01
reduced chi2:  5138.421551743697
***************************************************
Z2-amplitude:  1.34e+01
reduced chi2:  2987.217485337081
***************************************************
Z2-amplitude:  1.79e+00
reduced chi2:  5817.726537529166
***************************************************
Z2-amplitude:  1.43e+01
reduced chi2:  7647.915334127333
***************************************************
Z2-amplitude:  9.10e+00
reduced chi2:  5879.619480766213
***************************************************
Z2-amplitude:  9.45e+00
reduced chi2:  6813.4420476352525
***************************************************
Z2-amplitude:  9.58e+00
reduced chi2:  5451.938345974184
***************************************************
Z2-amplitude:  3.60e+01
reduced chi2:  4675.765876119452
***************************************************
Z2-amplitude:  1.61e+00
reduced chi2:  6741.402291648047
******************

In [199]:
amps = np.array([m_.values[0] for m_ in m1]) ## floating phase
phases = [m_.values[2] for m_ in m1]  ## floating phase

amps2 = np.array([m_.values[0] for m_ in m2]) ## fixed phase -2.8
phases2 = [m_.values[2] for m_ in m2]  ## fixed phase -2.8

In [200]:
_, ax = plt.subplots(1,2,figsize=(9.5,4))
ax[0].scatter(range(len(amps[10:])), amps[10:])
ax[0].scatter(range(len(amps[10:])), amps2[10:])
ax[1].scatter(range(len(amps[10:])), phases[10:])
ax[1].scatter(range(len(amps[10:])), phases2[10:])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
arrrange =  np.arange(100, 500 ,0.1)
residuals = [np.mean((amps2[10:]/scale_)%1) for scale_ in arrrange]
scale = arrrange[np.argmin(residuals)]
print(scale)

378.79999999998415


In [203]:
_,ax = plt.subplots(figsize=(9.5,4))
scale = 385

# Perrinf
ax.scatter(range(len(amps[10:])), amps[10:]/scale, label='floating-phase')
ax.scatter(range(len(amps[10:])), amps2[10:]/scale, label='fixed-phase')

ax.legend()
ax.set(xlabel='dataset', ylabel='Charges', ylim=(-10,2), xlim=(40,None))
ax.grid(axis='y')
ax.minorticks_on()
ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
ax.legend()
from matplotlib.ticker import MaxNLocator
ax.yaxis.set_major_locator(MaxNLocator(integer=True))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### fit files

In [110]:
ll = likelihood_analyser.LikelihoodAnalyser()

def getA(i):
# for i,bdf_ in enumerate(data_bdfs2):
    print(i,'  ******************')
    fname = r'/data/new_trap/20201116/Bead2/Discharge/Neutrality_20201120/Discharge_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)
    drive_freq = int(bb.cant_freq)
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_response,
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [-1000, 1000], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': True}

    frequency = fit_kwargs['f']

    xx2 = bb.response_at_freq2('z', frequency, bandwidth=bandwidth)
    xx2 = xx2[5000:-5000:10]  # cut out the first and last second

    m1_tmp = ll.find_mle_sin(xx2, fsamp=500, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)


    return m1_tmp.values[0]

In [127]:
len(range(0,21,2))

11

In [161]:
phis = np.arange(0,2*np.pi,0.2)
res_mean_all = []
res_std_all = []
for i in np.arange(0,21,0.1):
    res_mean4 = []
    res_std4 = []
    for phi_ in phis:
        phi_response = phi_
        res_tmp = np.array(Parallel(n_jobs=15)(delayed(getA)(i) for i in tqdm(range(int(i*1000), int(i*1000)+100))))
        res_mean4.append(np.mean(res_tmp))
        res_std4.append(np.std(res_tmp))
    res_mean_all.append(res_mean4)
    res_std_all.append(res_std4)




  0%|          | 0/100 [00:00<?, ?it/s]Exception ignored in: <Finalize object, dead>
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/util.py", line 186, in __call__
    res = self._callback(*self._args, **self._kwargs)
  File "/home/analysis_user/.local/lib/python3.6/site-packages/joblib/externals/loky/backend/synchronize.py", line 96, in _cleanup
    sem_unlink(name)
FileNotFoundError: [Errno 2] No such file or directory
Exception ignored in: <Finalize object, dead>
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/util.py", line 186, in __call__
    res = self._callback(*self._args, **self._kwargs)
  File "/home/analysis_user/.local/lib/python3.6/site-packages/joblib/externals/loky/backend/synchronize.py", line 96, in _cleanup
    sem_unlink(name)
FileNotFoundError: [Errno 2] No such file or directory
Exception ignored in: <Finalize object, dead>
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/

 90%|█████████ | 90/100 [00:00<00:00, 221.30it/s]


100%|██████████| 100/100 [00:00<00:00, 239.05it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 30%|███       | 30/100 [00:00<00:00, 202.45it/s]


 37%|███▋      | 37/100 [00:00<00:00, 111.26it/s]


 60%|██████    | 60/100 [00:00<00:00, 110.45it/s]


 90%|█████████ | 90/100 [00:00<00:00, 97.02it/s] 


100%|██████████| 100/100 [00:00<00:00, 109.52it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 30%|███       | 30/100 [00:00<00:00, 110.99it/s]


 45%|████▌     | 45/100 [00:00<00:00, 85.04it/s] 


 60%|██████    | 60/100 [00:00<00:00, 70.54it/s]


 75%|███████▌  | 75/100 [00:01<00:00, 71.07it/s]


 90%|█████████ | 90/100 [00:01<00:00, 68.63it/s]


100%|██████████| 100/100 [00:01<00:00, 77.03it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 30%|███       | 30/100 [00:00<00:00, 160.68it/s]


 60%|██████    | 60/100 [00:00<00:00, 154.99it/s]


 90%|█████████ | 90/100 [00:00<00:00, 135.83it/s]


100%|██████████| 100/100 [00:00<00:00, 145.

100%|██████████| 100/100 [00:00<00:00, 162.46it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 30%|███       | 30/100 [00:00<00:00, 153.05it/s]


 60%|██████    | 60/100 [00:00<00:00, 144.90it/s]


 90%|█████████ | 90/100 [00:00<00:00, 124.99it/s]


100%|██████████| 100/100 [00:00<00:00, 131.60it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 30%|███       | 30/100 [00:00<00:00, 203.68it/s]


 60%|██████    | 60/100 [00:00<00:00, 211.78it/s]


 90%|█████████ | 90/100 [00:00<00:00, 201.09it/s]


100%|██████████| 100/100 [00:00<00:00, 222.09it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 60%|██████    | 60/100 [00:00<00:00, 214.20it/s]


 90%|█████████ | 90/100 [00:00<00:00, 191.71it/s]


100%|██████████| 100/100 [00:00<00:00, 183.37it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 60%|██████    | 60/100 [00:00<00:00, 178.12it/s]


 90%|█████████ | 90/100 [00:00<00:00, 163.11it/s]


100%|██████████| 100/100 [00:00<00:00, 177.23it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 60%|

100%|██████████| 100/100 [00:00<00:00, 184.01it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 60%|██████    | 60/100 [00:00<00:00, 243.59it/s]


 90%|█████████ | 90/100 [00:00<00:00, 202.96it/s]


100%|██████████| 100/100 [00:00<00:00, 218.91it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 60%|██████    | 60/100 [00:00<00:00, 297.89it/s]


 90%|█████████ | 90/100 [00:00<00:00, 226.40it/s]


100%|██████████| 100/100 [00:00<00:00, 240.73it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 30%|███       | 30/100 [00:00<00:00, 287.91it/s]


 60%|██████    | 60/100 [00:00<00:00, 270.08it/s]


 90%|█████████ | 90/100 [00:00<00:00, 213.59it/s]


100%|██████████| 100/100 [00:00<00:00, 219.39it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 30%|███       | 30/100 [00:00<00:00, 247.89it/s]


 60%|██████    | 60/100 [00:00<00:00, 204.52it/s]


 90%|█████████ | 90/100 [00:00<00:00, 202.17it/s]


100%|██████████| 100/100 [00:00<00:00, 203.77it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 30%|

  0%|          | 0/100 [00:00<?, ?it/s]


 60%|██████    | 60/100 [00:00<00:00, 381.66it/s]


 90%|█████████ | 90/100 [00:00<00:00, 318.73it/s]


100%|██████████| 100/100 [00:00<00:00, 334.06it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 30%|███       | 30/100 [00:00<00:00, 83.08it/s]


 45%|████▌     | 45/100 [00:00<00:01, 52.70it/s]


 60%|██████    | 60/100 [00:01<00:00, 40.22it/s]


 75%|███████▌  | 75/100 [00:02<00:00, 31.99it/s]


 90%|█████████ | 90/100 [00:02<00:00, 26.81it/s]


100%|██████████| 100/100 [00:02<00:00, 34.09it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 60%|██████    | 60/100 [00:00<00:00, 243.70it/s]


 90%|█████████ | 90/100 [00:00<00:00, 213.83it/s]


100%|██████████| 100/100 [00:00<00:00, 228.81it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 60%|██████    | 60/100 [00:00<00:00, 203.14it/s]


 90%|█████████ | 90/100 [00:00<00:00, 177.03it/s]


100%|██████████| 100/100 [00:00<00:00, 189.36it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 30%|███    

...

 60%|██████    | 60/100 [00:00<00:00, 229.29it/s]


 90%|█████████ | 90/100 [00:00<00:00, 182.97it/s]


100%|██████████| 100/100 [00:00<00:00, 188.02it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 60%|██████    | 60/100 [00:00<00:00, 202.55it/s]


 90%|█████████ | 90/100 [00:00<00:00, 176.83it/s]


100%|██████████| 100/100 [00:00<00:00, 190.20it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 60%|██████    | 60/100 [00:00<00:00, 186.78it/s]


 90%|█████████ | 90/100 [00:00<00:00, 196.61it/s]


100%|██████████| 100/100 [00:00<00:00, 216.46it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 60%|██████    | 60/100 [00:00<00:00, 241.45it/s]


 90%|█████████ | 90/100 [00:00<00:00, 210.99it/s]


100%|██████████| 100/100 [00:00<00:00, 222.98it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 60%|██████    | 60/100 [00:00<00:00, 239.28it/s]


 90%|█████████ | 90/100 [00:00<00:00, 191.40it/s]


100%|██████████| 100/100 [00:00<00:00, 203.82it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 60%|

 60%|██████    | 60/100 [00:00<00:00, 234.48it/s]


 90%|█████████ | 90/100 [00:00<00:00, 171.28it/s]


100%|██████████| 100/100 [00:00<00:00, 181.56it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 30%|███       | 30/100 [00:00<00:00, 268.65it/s]


 60%|██████    | 60/100 [00:00<00:00, 250.50it/s]


 90%|█████████ | 90/100 [00:00<00:00, 194.40it/s]


100%|██████████| 100/100 [00:00<00:00, 200.32it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 60%|██████    | 60/100 [00:00<00:00, 233.89it/s]


 90%|█████████ | 90/100 [00:00<00:00, 180.56it/s]


100%|██████████| 100/100 [00:00<00:00, 191.54it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 60%|██████    | 60/100 [00:00<00:00, 236.03it/s]


 90%|█████████ | 90/100 [00:00<00:00, 181.10it/s]


100%|██████████| 100/100 [00:00<00:00, 194.89it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 60%|██████    | 60/100 [00:00<00:00, 217.85it/s]


 90%|█████████ | 90/100 [00:00<00:00, 184.33it/s]


100%|██████████| 100/100 [00:00<00:00, 197.89it/

 60%|██████    | 60/100 [00:00<00:00, 243.38it/s]


 90%|█████████ | 90/100 [00:00<00:00, 195.88it/s]


100%|██████████| 100/100 [00:00<00:00, 209.06it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 60%|██████    | 60/100 [00:00<00:00, 273.01it/s]


 90%|█████████ | 90/100 [00:00<00:00, 189.44it/s]


100%|██████████| 100/100 [00:00<00:00, 198.73it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 60%|██████    | 60/100 [00:00<00:00, 241.92it/s]


 90%|█████████ | 90/100 [00:00<00:00, 198.35it/s]


100%|██████████| 100/100 [00:00<00:00, 212.75it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 60%|██████    | 60/100 [00:00<00:00, 207.87it/s]


 90%|█████████ | 90/100 [00:00<00:00, 201.57it/s]


100%|██████████| 100/100 [00:00<00:00, 219.84it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 60%|██████    | 60/100 [00:00<00:00, 241.39it/s]


 90%|█████████ | 90/100 [00:00<00:00, 203.71it/s]


100%|██████████| 100/100 [00:00<00:00, 214.32it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 60%|

  0%|          | 0/100 [00:00<?, ?it/s]


 60%|██████    | 60/100 [00:00<00:00, 241.07it/s]


 90%|█████████ | 90/100 [00:00<00:00, 193.64it/s]


100%|██████████| 100/100 [00:00<00:00, 204.31it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 60%|██████    | 60/100 [00:00<00:00, 231.33it/s]


 90%|█████████ | 90/100 [00:00<00:00, 187.42it/s]


100%|██████████| 100/100 [00:00<00:00, 197.98it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 60%|██████    | 60/100 [00:00<00:00, 263.97it/s]


 90%|█████████ | 90/100 [00:00<00:00, 178.98it/s]


100%|██████████| 100/100 [00:00<00:00, 188.69it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 60%|██████    | 60/100 [00:00<00:00, 222.31it/s]


 90%|█████████ | 90/100 [00:00<00:00, 191.18it/s]


100%|██████████| 100/100 [00:00<00:00, 203.75it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 60%|██████    | 60/100 [00:00<00:00, 244.21it/s]


 90%|█████████ | 90/100 [00:00<00:00, 188.21it/s]


100%|██████████| 100/100 [00:00<00:00, 196.32it/s]


  0%|

100%|██████████| 100/100 [00:00<00:00, 223.17it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 30%|███       | 30/100 [00:00<00:00, 289.72it/s]


 60%|██████    | 60/100 [00:00<00:00, 275.85it/s]


 90%|█████████ | 90/100 [00:00<00:00, 193.69it/s]


100%|██████████| 100/100 [00:00<00:00, 198.62it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 60%|██████    | 60/100 [00:00<00:00, 241.90it/s]


 90%|█████████ | 90/100 [00:00<00:00, 205.21it/s]


100%|██████████| 100/100 [00:00<00:00, 221.93it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 60%|██████    | 60/100 [00:00<00:00, 194.37it/s]


 90%|█████████ | 90/100 [00:00<00:00, 201.03it/s]


100%|██████████| 100/100 [00:00<00:00, 216.17it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 60%|██████    | 60/100 [00:00<00:00, 212.06it/s]


 90%|█████████ | 90/100 [00:00<00:00, 210.93it/s]


100%|██████████| 100/100 [00:00<00:00, 231.96it/s]


  0%|          | 0/100 [00:00<?, ?it/s]


 60%|██████    | 60/100 [00:00<00:00, 257.98it/s]


 90%|

In [172]:
_,ax = plt.subplots()
ax.errorbar(phis, np.array(res_mean_all[0])/scale, yerr=np.array(res_std_all[0])/np.sqrt(100)/scale, fmt='o')
ax.errorbar(phis, np.array(res_mean_all[4])/scale, yerr=np.array(res_std_all[4])/np.sqrt(100)/scale, fmt='o')
ax.errorbar(phis, np.array(res_mean_all[-1])/scale, yerr=np.array(res_std_all[-1])/np.sqrt(100)/scale, fmt='o')
ax.plot(phis, phis*0, 'r--')
ax.set(xlabel='phase [rad]', ylabel=r'$\epsilon$')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0.5, 0, 'phase [rad]'), Text(0, 0.5, '$\\epsilon$')]

In [132]:
np.argmin(res_mean_all[0])

15

In [182]:
phases = np.array([phis[np.argmin(res_)] for res_ in res_mean_all])
amps = np.array([np.std(res_)*np.sqrt(2)/scale for res_ in res_mean_all])
amps0 = np.array([res_[0]/scale for res_ in res_mean_all])
std0 = np.array([res_[0]/scale/np.sqrt(100) for res_ in res_std_all])

In [188]:
_,ax = plt.subplots(2,1,figsize=(9.2,7), sharex=True)
ax[0].plot(np.arange(0,21,0.1)*1000, phases, '.--')
ax[0].set(ylabel='phase [rad]')
ax[1].plot(np.arange(0,21,0.1)*1000, amps , '.', label='amplitude')
ax[1].plot(np.arange(0,21,0.1)*1000, amps0, '.', label='response at phase=0')
ax[1].plot(np.arange(0,21,0.1)*1000, np.arange(0,21,0.1)*0, 'r--')
ax[0].set(ylabel='phase [rad]')
ax[1].set(xlabel='dataset', ylabel=r'$\epsilon$')
ax[1].legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [186]:
_,ax = plt.subplots(1,2,figsize=(9.2,4))
ax[0].scatter(phases, amps0)
ax[0].plot(phases, phases*0, 'r--')
ax[0].set(xlabel='phase [rad]', ylabel=r'$\epsilon$')
ax[1].hist(phases, bins=10)
ax[1].set(xlabel='phase [rad]')
print(np.mean(std0))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.005755975864607641


In [119]:
_,ax = plt.subplots()
ax.errorbar(phis, np.array(res_mean)/scale, yerr=np.array(res_std)/np.sqrt(1000)/scale, fmt='o')
ax.errorbar(phis, np.array(res_mean2)/scale, yerr=np.array(res_std2)/np.sqrt(1000)/scale, fmt='o')
ax.errorbar(phis, np.array(res_mean3)/scale, yerr=np.array(res_std3)/np.sqrt(1000)/scale, fmt='o')
ax.errorbar(phis, np.array(res_mean4)/scale, yerr=np.array(res_std4)/np.sqrt(1000)/scale, fmt='o')
ax.plot(phis, phis*0, 'r--')
ax.plot(2.8, np.mean(res_tmp_2_8)/scale, '*')
ax.plot(1.32, np.mean(res_tmp_1_32)/scale, '*')
ax.plot(0, np.mean(res_tmp_0)/scale, '*')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [78]:
phi0 = 0
phi_response = phi0
res_tmp_0 = np.array(Parallel(n_jobs=40)(delayed(getA)(i) for i in tqdm(range(20000))))

100%|██████████| 20000/20000 [13:14<00:00, 24.45it/s]


In [102]:
_,ax = plt.subplots()
# ax.hist(res_tmp/scale, bins=30)
# ax.hist(res_tmp_2_8/scale, bins=30)
# ax.hist(res_tmp_1_32/scale, bins=30, alpha=0.7)
ax.hist(means/scale, bins=30, alpha=0.5)
print(np.mean(res_tmp)/scale, np.std(res_tmp)/scale/np.sqrt(10000))
print(np.mean(res_tmp_2_8)/scale, np.std(res_tmp_2_8)/scale/np.sqrt(20000))
print(np.mean(res_tmp_1_32)/scale, np.std(res_tmp_1_32)/scale/np.sqrt(20000))
print(np.mean(res_tmp_0)/scale, np.std(res_tmp_0)/scale/np.sqrt(20000))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.0006805222121827159 0.0005603446164143654
-0.0009786831689251167 0.00040998856018796195
0.0008856047160216867 0.00041128438784253477
0.001249881834284009 0.0004087561899279354


In [85]:
tmpp = res_tmp_2_8
means = np.array([np.mean(tmpp[j*100:(j+1)*100]) for j in range(200)])
stds = np.array([np.std(tmpp[j*100:(j+1)*100]) for j in range(200)])

In [91]:
_,ax = plt.subplots(figsize=(9.5,4))
ax.errorbar(np.arange(0,20000,100), means/scale, yerr=stds/scale/np.sqrt(100),fmt='o')
ax.plot(np.arange(0,20000,100), np.arange(0,20000,100)*0, 'r--')
ax.set(xlabel='dataset', ylabel=r'$\epsilon$')
z, cov = np.polyfit(np.arange(0,20000,100), means/scale, 0, cov=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [95]:
z, cov, np.mean(res_mean)/scale, np.mean(res_tmp_2_8)/scale

(array([-0.00097868]),
 array([[1.94744396e-07]]),
 3.213455867827143e-05,
 -0.0009786831689251167)

### Profile Likelihood